In [1]:
from scipy.io import netcdf
from matplotlib import pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap
import calculations as calc
import similarity_measures

In [2]:
nc = netcdf.netcdf_file("data/era-int_1979-2019-mm-l70-u.nc")

In [3]:
u = nc.variables["u"]

my_qbo = calc.derive(u[:,:,:,:], lon = 148, lat = 126, level = 0, lon_step = 0, lat_step = 0)
my_deseasonalized_qbo = calc.deseasonalize_time_series(my_qbo)
my_deseasonalized_qbo

[0.6706436618915272,
 0.9066453521815323,
 0.13951518723756864,
 0.019025036494679848,
 0.02998170114413044,
 0.09736790291641667,
 -0.23508083349102457,
 -0.5182837261201245,
 -0.032327627782482854,
 -0.7724871277071864,
 -0.289251924554193,
 -0.44204395247792255,
 -0.8964192336548233,
 -1.1186582819422368,
 -0.9685367471896824,
 -0.5567934859232352,
 -0.6285058788338547,
 -0.5187048965135825,
 -0.12152283012302104,
 1.011516789486432,
 0.26156869806799565,
 0.4539633219548187,
 0.11220072154552126,
 0.465329854990965,
 0.15061690071758319,
 -0.2570599874253456,
 1.6287539234059634,
 0.7526508974656692,
 0.6873745217724916,
 -0.010885301680638696,
 0.9365577883558083,
 0.04092754858735435,
 0.4085220520453482,
 -1.1506183203801794,
 0.1369834576531038,
 -0.0053672158127868195,
 -0.22965331745340778,
 -0.47808179362730135,
 -0.35407643849699266,
 -0.18502632795604562,
 0.571259706829966,
 -0.42800345483233115,
 -2.16888676654629,
 -2.064055551046519,
 -1.531431575023435,
 -0.8961952621

In [ ]:
sim = calc.calculate_series_similarity_per_period(u[:, :, :, :], my_deseasonalized_qbo, level=0, period_length=12, sim_func=similarity_measures.correlation_similarity)

In [ ]:
#Plot similarity on maps
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1, clip=True)
mapper= matplotlib.cm.ScalarMappable(norm=norm)

fig, axes =plt.subplots(figsize = (20,30), nrows=10, ncols=4)

for i, ax in enumerate(axes.flat):
    m = Basemap(projection='mill',lon_0=30,resolution='l', ax=ax)
    m.drawcoastlines()

    lons, lats = m.makegrid(512, 256)
    x, y = m(lons, lats)

    cs = m.contourf(x, y, sim[i])
    ax.set_title(str(1979 + i))
    
fig.suptitle("Correlation to qbo from 1979-2018")
plt.colorbar(mapper, cax=plt.axes([1, 0.1, 0.045, 0.78]))
plt.show()